In [23]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib import rc
from sklearn.metrics import r2_score
rc('font', family = 'AppleGothic')

In [14]:
data = pd.read_csv("oecd_family.csv")
#합계출산율(Total fertility rate) 추출
birth_rate = data.loc[(data.IND=='FAM1',['Country', 'Year','Value'])]

#2021년 이상 데이터 삭제
birth_rate = birth_rate.drop(birth_rate.loc[birth_rate.Year>=2021].index)

#2012년부터 2020년까지 출산율 결측치 없음
birth_rate.isna().sum()

#인덱스 재정렬
birth_rate.index = range(0,len(birth_rate))

#한글화를 위한 리스트 생성
CountryName = ["이스라엘","룩셈부르크","네덜란드","그리스","아이슬란드","라트비아","슬로베니아","프랑스","폴란드","튀르키예","스페인","벨기에","멕시코","아일랜드","스위스","핀란드","칠레","캐나다","대한민국","뉴질랜드","오스트레일리아","헝가리","미국","영국","오스트리아","스웨덴","덴마크","슬로바키아","체코","콜롬비아","일본","노르웨이","코스타리카","리투아니아","독일","에스토니아","포르투갈","이탈리아"]

#영어이름을 한글로
for i in range(0,38):
    for j in range(0,9):
        birth_rate.loc[i*9:i*9+j,"Country"] = CountryName[i]
        
birth_rate

,Country,Year,Value
0,이스라엘,2012,3.05
1,이스라엘,2013,3.03
2,이스라엘,2014,3.08
3,이스라엘,2015,3.09
4,이스라엘,2016,3.11
...,...,...,...
337,이탈리아,2016,1.36
338,이탈리아,2017,1.34
339,이탈리아,2018,1.31
340,이탈리아,2019,1.27


In [16]:
#경제활동인구(여자)중 경제활동 참가율 data
df2 = pd.read_csv("경제활동인구 및 참가율(OECD).csv", encoding= "euc-kr", skiprows = 1)
plt.rcParams['font.family'] = 'NanumGothicOTF'

#column(열) 이름 바꾸기 = 연도
    
df2= df2.rename(columns = {'여자 (%)' : '2012'})
for i in range(1,10):
    df2 = df2.rename(columns = {'여자 (%).{}'.format(i): str(2012+i)} )

namelist = []
#row(행) 이름 바꾸기. = 국가    
for i in range(len(df2)):
    x = df2.loc[i,'국가별'].find("(")
    name = df2.loc[i,'국가별']
    if(x != -1):
        name = name[:x]
        namelist.append(name)
        df2.rename(index = {i :name},inplace = True)
    else:
        df2.rename(index = {i :name},inplace = True)


#필요한 데이터만 추출.
col_name = []
for i in range(2012,2021):
    col_name.append(str(i))

rate_df = df2[col_name]

#NaN 값 제거.
rate_df.drop(rate_df.index[0], inplace= True)
rate_df.dropna(inplace=True)
rate_df.head()

#빈값 찾기.
col = rate_df.columns.values

index = rate_df.index.values
outlier = {}
for i in index:
    for c in col:
        if (rate_df.loc[i,str(c)] == '-'):
            outlier[c] = i

#국가별 평균값으로 빈값 찾기.
sum = 0
for key in outlier.keys():
    sum = 0
    for c in col:
        if (c == key):
            continue
        else:
            sum += float(rate_df.loc[outlier.get(key),str(c)])
    rate_df.loc[outlier.get(key), key] = round(sum/(len(col)-len(outlier)),1)

country = []
year = []
value = []
rate_val = []

#새로운 데이터 만들기 위한 값 배치.
for i in range(len(namelist)):
    for j in range(2012,2021):
        country.append(namelist[i])
        year.append(j)

for i in range(len(country)):
    rate_val.append(float(rate_df[str(year[i])][country[i]]))

# 전처리 완료한 데이터.
rate_df = pd.DataFrame({
    'Country' : country,
    'Year' : year,
    'rate_val' : rate_val
})

/var/folders/ls/8g_ybb8s6493dzrgbbh9shjc0000gn/T/ipykernel_81975/2645756960.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_df.drop(rate_df.index[0], inplace= True)
/var/folders/ls/8g_ybb8s6493dzrgbbh9shjc0000gn/T/ipykernel_81975/2645756960.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_df.dropna(inplace=True)


In [17]:
#gdp 데이터
df = pd.read_csv("OECD 국가의 주요지표.csv", encoding= "euc-kr", skiprows = 1)

#column(열) 이름 바꾸기 = 연도
df= df.rename(columns = {'국내총생산(GDP)(당해년 가격) (10억US$)' : '2012'})

for i in range(1,10):
    df = df.rename(columns = {'국내총생산(GDP)(당해년 가격) (10억US$).{}'.format(i): str(2012+i)} )

#row(행) 이름 바꾸기. = 국가
for i in range(len(df)):
    df.rename(index = {i :df.loc[i,'국가별']},inplace = True)

#필요한 GDP 데이터만 데이터프레임으로 추출.
#column 이름으로 추출.
col_name = []
for i in range(2012,2021):
    col_name.append(str(i))

GDP_df = df[col_name]

#NaN 값 제거.
GDP_df.drop(GDP_df.index[0], inplace= True)
GDP_df.dropna(inplace=True)

#OECD 총값 제거.
GDP_df.drop(GDP_df.tail(1).index,inplace=True)
#빈값, 잘못된 값 찾기
col = GDP_df.columns.values

index = GDP_df.index.values
outlier = {}
for i in index:
    for c in col:
        if (GDP_df.loc[i,str(c)] == '-'):
            outlier[c] = i
#빈값, 잘못된 값을 각 국의 평균값으로 대체.
sum = 0
for key in outlier.keys():
    sum = 0
    for c in col:
        if (c == key):
            continue
        else:
            sum += float(GDP_df.loc[outlier.get(key),str(c)])
    GDP_df.loc[outlier.get(key), key] = round(sum/(len(col)-len(outlier)),1)

country = []
year = []
value = []
gdp_val = []

#새로운 데이터 만들기 위한 값 배치.
for i in range(len(namelist)):
    for j in range(2012,2021):
        country.append(namelist[i])
        year.append(j)

for i in range(len(country)):
    gdp_val.append(float(GDP_df[str(year[i])][country[i]]))
    
#전처리 완료한 데이터.
gdp_df = pd.DataFrame({
    'Country' : country,
    'Year' : year,
    'gdp_val' : gdp_val
})

gdp_df

/var/folders/ls/8g_ybb8s6493dzrgbbh9shjc0000gn/T/ipykernel_81975/1515712362.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GDP_df.drop(GDP_df.index[0], inplace= True)
/var/folders/ls/8g_ybb8s6493dzrgbbh9shjc0000gn/T/ipykernel_81975/1515712362.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GDP_df.dropna(inplace=True)
/var/folders/ls/8g_ybb8s6493dzrgbbh9shjc0000gn/T/ipykernel_81975/1515712362.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,Country,Year,gdp_val
0,대한민국,2012,1278.0
1,대한민국,2013,1370.6
2,대한민국,2014,1484.0
3,대한민국,2015,1465.3
4,대한민국,2016,1500.0
...,...,...,...
337,뉴질랜드,2016,188.9
338,뉴질랜드,2017,207.0
339,뉴질랜드,2018,212.2
340,뉴질랜드,2019,209.1


In [18]:
#성불평등지수
inequality = pd.read_csv("sex_inequality.csv")
inequality = inequality.set_index('국가별')
inequality = inequality.drop(['대륙'], axis=1)

country = []
year = []
value = []
inequality_val = []

#새로운 데이터 만들기 위한 값 배치.
for i in range(len(namelist)):
    for j in range(2012,2021):
        country.append(namelist[i])
        year.append(j)

for i in range(len(country)):
    inequality_val.append(float(inequality[str(year[i])][country[i]]))

# 전처리 완료한 데이터.
inequality = pd.DataFrame({
    'Country' : country,
    'Year' : year,
    'inequality_val' : inequality_val
})
inequality

,Country,Year,inequality_val
0,대한민국,2012,0.108
1,대한민국,2013,0.108
2,대한민국,2014,0.100
3,대한민국,2015,0.096
4,대한민국,2016,0.086
...,...,...,...
337,뉴질랜드,2016,0.126
338,뉴질랜드,2017,0.112
339,뉴질랜드,2018,0.102
340,뉴질랜드,2019,0.100


In [19]:
totaldf = pd.merge(birth_rate, rate_df, on = ['Country','Year'])
totaldf = pd.merge(totaldf, gdp_df, on = ['Country','Year'])
totaldf = pd.merge(totaldf, inequality, on = ['Country','Year'])

In [25]:
#그래프가 많이 나와서 모두 주석처리 해놨습니다.

In [24]:
#국가별 회귀(gdp_val)

# for i in namelist:
#     d =  totaldf[totaldf['Country']== i]
#     d.plot(kind = "scatter",x = 'gdp_val', y = 'Value')
#     fit_weight = np.polyfit(d['gdp_val'], d['Value'],1) #d['gdp_val']를 x값으로, d['Value']를 y값으로 하여 1차식으로 피팅.
#     trend_f = np.poly1d(fit_weight) #weight를 이용해 1차 함수 반환.
#     # r2(결정계수): 회귀 모델에서 독립변수가 종속 변수를 얼마만큼 설명해 주는지를 가리키는 지표.
#     r2 = r2_score(totaldf['Value'], trend_f(totaldf['gdp_val']))
#     plt.plot(d['gdp_val'], trend_f(d['gdp_val']),"r-")
#     plt.title("{} y = {:.6f}x + ({:.6f}) [R2 = {:.6f}]".format(i,fit_weight[0], fit_weight[1], r2))


In [219]:
#국가별 회귀(rate_val)
# for i in namelist:
#     d =  totaldf[totaldf['Country']== i]
#     d.plot(kind = "scatter",x = 'rate_val', y = 'Value')
#     fit_weight = np.polyfit(d['rate_val'], d['Value'],1)
#     trend_f = np.poly1d(fit_weight)
#     r2 = r2_score(totaldf['Value'], trend_f(totaldf['rate_val']))
#     plt.plot(d['rate_val'], trend_f(d['rate_val']),"r-")
#     plt.title("{} y = {:.6f}x + ({:.6f}) [R2 = {:.6f}]".format(i,fit_weight[0], fit_weight[1], r2))

In [221]:
#국가별 회귀(inequality_val)
# for i in namelist:
#     d =  totaldf[totaldf['Country']== i]
#     d.plot(kind = "scatter",x = 'inequality_val', y = 'Value')
#     fit_weight = np.polyfit(d['inequality_val'], d['Value'],1)
#     r2 = r2_score(totaldf['Value'], trend_f(totaldf['inequality_val']))
#     trend_f = np.poly1d(fit_weight)
#     plt.plot(d['inequality_val'], trend_f(d['inequality_val']),"r-")
#     plt.title("{} y = {:.6f}x + ({:.6f}) [R2 = {:.6f}]".format(i,fit_weight[0], fit_weight[1], r2))

In [232]:
#연도별 회귀 (gdp)
# for i in range(2012,2021):
#     d =  totaldf[totaldf['Year']== i]
#     d = d[d['Country']!='미국'] #미국의 gdp값이 과도하게 크므로 제외함.
#     d.plot(kind = "scatter",x = 'gdp_val', y = 'Value')
#     fit_weight = np.polyfit(d['gdp_val'],d['Value'],1)
#     trend_f = np.poly1d(fit_weight)
#     r2 = r2_score(totaldf['Value'], trend_f(totaldf['gdp_val']))
#     plt.plot(d['gdp_val'], trend_f(d['gdp_val']),"r-")
#     plt.title("{}년 y = {:.6f}x + ({:.6f}) [R2 = {:.6f}]".format(i,fit_weight[0], fit_weight[1], r2))

In [229]:
#연도별 회귀 (rate)
# for i in range(2012,2021):
#     d =  totaldf[totaldf['Year']== i]
#     d.plot(kind = "scatter",x = 'rate_val', y = 'Value')
#     fit_weight = np.polyfit(d['rate_val'],d['Value'],1)
#     trend_f = np.poly1d(fit_weight)
#     r2 = r2_score(totaldf['Value'], trend_f(totaldf['rate_val']))
#     plt.plot(d['rate_val'], trend_f(d['rate_val']),"r-")
#     plt.title("{}년 y = {:.6f}x + ({:.6f}) [R2 = {:.6f}]".format(i,fit_weight[0], fit_weight[1], r2))

In [8]:
# 연도별 회귀 (inequality)
# for i in range(2012,2021):
#     d =  totaldf[totaldf['Year']== i]
#     d.plot(kind = "scatter",x = 'inequality_val', y = 'Value')
#     fit_weight = np.polyfit(d['inequality_val'],d['Value'],1)
#     trend_f = np.poly1d(fit_weight)
#     r2 = r2_score(totaldf['Value'], trend_f(totaldf['inequality_val']))
#     plt.plot(d['inequality_val'], trend_f(d['inequality_val']),"r-")
#     plt.title("{}년 y = {:.6f}x + ({:.6f}) [R2 = {:.6f}]".format(i,fit_weight[0], fit_weight[1], r2))